In [49]:
import pandas as pd

In [56]:
sip_dim_store_data = pd.read_csv('SOURCES_sip_dim_store.csv')
sip_dim_store_data

,store_id,store_name,surface,format_id,commune,opening_date,is_active,sanitary_resolution,process_room,frogmi_id,frogmi_is_active,baking_trays_for_bakery
0,67,CASABLANCA,550,1,CASA BLANCA,01-10-2008,True,False,True,e8b53fa4-b464-42ba-a041-ffc4a32c244c,False,12
1,328,GRAN AVENIDA I,1316,1,SAN MIGUEL,01-01-2008,True,True,True,a73f9e16-b44f-4e98-85b4-3c1a489f109c,False,12
2,332,1 NORTE,843,1,VIÃ‘A DEL MAR,22-06-2016,True,True,True,42528cbf-43c7-40d1-83b4-abea2e2c3f62,False,18
3,486,CIUDAD DE LOS VALLES,1156,1,PUDAHUEL,01-12-2009,True,True,True,8426398d-772b-410e-a36b-d5c32d529c94,False,18
4,978,SAN JOAQUIN,2100,1,SAN JOAQUIN,21-09-2012,True,True,True,de9184e3-ce3c-4eeb-83c4-48d3070b7ac6,False,15


In [98]:
df = sip_dim_store_data.copy()
filter_by = 'store_id'
store_id = 67
filtered_df = df.loc[df[filter_by] == store_id]
headers = filtered_df.columns
values = filtered_df.values[0]
store_params = dict(zip(headers, values))
store_params

{'store_id': 67,
 'store_name': 'CASABLANCA',
 'surface': 550,
 'format_id': 1,
 'commune': 'CASA BLANCA',
 'opening_date': '01-10-2008',
 'is_active': True,
 'sanitary_resolution': False,
 'process_room': True,
 'frogmi_id': 'e8b53fa4-b464-42ba-a041-ffc4a32c244c',
 'frogmi_is_active': False,
 'baking_trays_for_bakery': 12}

In [99]:
if store_params['sanitary_resolution'] and store_params['process_room']:
    print('Se puede producir cualquier sku, pero se utiliza Pollo fresco y Pollo congelado')
    productos = [858091, 571581, 638484, 626218, 632861, 638771, 966668, 966669, 966670, 966679]
elif not store_params['sanitary_resolution'] and store_params['process_room']:
    print('Solo se puede producir pollo pre-horneado')
    productos = [655130, 626218, 632861, 638771, 966668, 966669, 966670, 966679]
else:
    productos = []
    print('No se puede producir')
products_list = productos
products_list

Solo se puede producir pollo pre-horneado


[655130, 626218, 632861, 638771, 966668, 966669, 966670, 966679]

In [100]:
products_data = pd.read_csv('SOURCES_sip_dim_products.csv', sep=',')
products_to_produce = products_data.loc[products_data['sku'].isin(products_list)][['sku_name', 'sku', 'group_id', 'type']]
products_to_produce

,sku_name,sku,group_id,type
2,PAN COLIZA CORRIENTE AMADA MASA KG,626218,170020302,1
3,PAN HALLULLA PLANA LAF KG,632861,170020104,1
4,PAN CIABATTA AMADA MASA KG,638771,170020302,1
6,POLLO ASADO LISTO TRADICIONAL TM UN.,655130,172011201,2
7,PAN HALLULLA AMADA MASA KG,966668,170020104,1
8,PAN MARRAQUETA AMADA MASA KG,966669,170020102,1
9,PAN DOBLADA AMADA MASA KG,966670,170020302,1
10,PAN HALLULLA INTEGRAL AMADA MASA KG,966679,170020302,1


In [164]:
predictions_data = pd.read_csv('BY_DemandPrediction_Day_COMP_202312262109.csv', sep=';')
prediction_data = predictions_data.loc[predictions_data['store_id'] == store_id]
prediction_data.columns = ['sku', 'store_id', 'predictionday', 'referenceday', 'finalforecast']
prediction_data

,sku,store_id,predictionday,referenceday,finalforecast
0,966679,67,2023-12-05,2023-12-01,3.0
1,966668,67,2023-12-06,2023-12-01,57.0
2,966669,67,2023-12-06,2023-12-01,57.0
3,966670,67,2023-12-06,2023-12-01,8.0
4,632861,67,2023-12-03,2023-12-01,1.0
...,...,...,...,...,...
220,966668,67,2023-12-09,2023-12-01,69.0
221,966669,67,2023-12-21,2023-12-01,56.0
222,966670,67,2023-12-21,2023-12-01,7.0
223,966670,67,2023-12-05,2023-12-01,8.0


In [165]:
merged_data = products_to_produce.merge(prediction_data, on='sku',how='left')
merged_data

,sku_name,sku,group_id,type,store_id,predictionday,referenceday,finalforecast
0,PAN COLIZA CORRIENTE AMADA MASA KG,626218,170020302,1,67,2023-12-20,2023-12-01,2.0
1,PAN COLIZA CORRIENTE AMADA MASA KG,626218,170020302,1,67,2023-12-10,2023-12-01,3.0
2,PAN COLIZA CORRIENTE AMADA MASA KG,626218,170020302,1,67,2023-12-26,2023-12-01,1.0
3,PAN COLIZA CORRIENTE AMADA MASA KG,626218,170020302,1,67,2023-12-04,2023-12-01,3.0
4,PAN COLIZA CORRIENTE AMADA MASA KG,626218,170020302,1,67,2023-12-11,2023-12-01,2.0
...,...,...,...,...,...,...,...,...
195,PAN HALLULLA INTEGRAL AMADA MASA KG,966679,170020302,1,67,2023-12-07,2023-12-01,3.0
196,PAN HALLULLA INTEGRAL AMADA MASA KG,966679,170020302,1,67,2023-12-08,2023-12-01,3.0
197,PAN HALLULLA INTEGRAL AMADA MASA KG,966679,170020302,1,67,2023-12-15,2023-12-01,3.0
198,PAN HALLULLA INTEGRAL AMADA MASA KG,966679,170020302,1,67,2023-12-01,2023-12-01,3.0


In [166]:
merged_filtered_data = merged_data.loc[merged_data['predictionday'] == merged_data['referenceday']]
merged_filtered_data

,sku_name,sku,group_id,type,store_id,predictionday,referenceday,finalforecast
11,PAN COLIZA CORRIENTE AMADA MASA KG,626218,170020302,1,67,2023-12-01,2023-12-01,3.0
43,PAN HALLULLA PLANA LAF KG,632861,170020104,1,67,2023-12-01,2023-12-01,1.0
57,PAN CIABATTA AMADA MASA KG,638771,170020302,1,67,2023-12-01,2023-12-01,22.0
89,POLLO ASADO LISTO TRADICIONAL TM UN.,655130,172011201,2,67,2023-12-01,2023-12-01,6.0
119,PAN HALLULLA AMADA MASA KG,966668,170020104,1,67,2023-12-01,2023-12-01,63.0
142,PAN MARRAQUETA AMADA MASA KG,966669,170020102,1,67,2023-12-01,2023-12-01,71.0
165,PAN DOBLADA AMADA MASA KG,966670,170020302,1,67,2023-12-01,2023-12-01,9.0
198,PAN HALLULLA INTEGRAL AMADA MASA KG,966679,170020302,1,67,2023-12-01,2023-12-01,3.0


In [167]:
bakery = merged_filtered_data.loc[merged_filtered_data['type'] == 1]
bakery

,sku_name,sku,group_id,type,store_id,predictionday,referenceday,finalforecast
11,PAN COLIZA CORRIENTE AMADA MASA KG,626218,170020302,1,67,2023-12-01,2023-12-01,3.0
43,PAN HALLULLA PLANA LAF KG,632861,170020104,1,67,2023-12-01,2023-12-01,1.0
57,PAN CIABATTA AMADA MASA KG,638771,170020302,1,67,2023-12-01,2023-12-01,22.0
119,PAN HALLULLA AMADA MASA KG,966668,170020104,1,67,2023-12-01,2023-12-01,63.0
142,PAN MARRAQUETA AMADA MASA KG,966669,170020102,1,67,2023-12-01,2023-12-01,71.0
165,PAN DOBLADA AMADA MASA KG,966670,170020302,1,67,2023-12-01,2023-12-01,9.0
198,PAN HALLULLA INTEGRAL AMADA MASA KG,966679,170020302,1,67,2023-12-01,2023-12-01,3.0


In [168]:
roastery = merged_filtered_data.loc[merged_filtered_data['type'] == 2]
roastery

,sku_name,sku,group_id,type,store_id,predictionday,referenceday,finalforecast
89,POLLO ASADO LISTO TRADICIONAL TM UN.,655130,172011201,2,67,2023-12-01,2023-12-01,6.0


In [108]:
def convert_int_to_char_str(integer: str, chars : int) -> str:
    """Converts an integer to a 4-character string.

    Args:
    integer: The integer to convert.

    Returns:
    A N-character string representation of the integer.
    """
    if isinstance(integer, int):
        integer_str = str(integer)
    else:
        integer_str = integer
    
    if len(integer_str) < chars:
        integer_str = integer_str.zfill(chars)
    return integer_str

In [169]:
from datetime import datetime
products_unique_array = products_data['sku'].astype(str).unique()
products_unique_list = [convert_int_to_char_str(sku, 18) for sku in products_unique_array]
products_unique_str  = ", ".join([f"'{value}'" for value in products_unique_list])
stores_id = [67]
str_store_id = ", ".join([f"'{convert_int_to_char_str(store_id, 4)}'" for store_id in stores_id])
current_date = datetime.now().strftime("%Y-%m-%d")
current_date = '2023-12-01'
stock_query = f"""
        SELECT
          DSKU.SKU_PRODUCT AS sku,
          LAOLS.DATE_VALUE AS fecha_medicion_inventario,
          CASE
          WHEN ROUND(SUM(LAOLS.NBR_ITM)) < 0 THEN 0
          ELSE ROUND(SUM(LAOLS.NBR_ITM))
          END AS stock_umb
        FROM `cl-data-procesada.PROD_AMP_BASE.LOGISTIC_ACTIVITY_IVNT_ANL` LAOLS
          INNER JOIN `cl-data-procesada.AMP.DIM_OU_HIERARCHY` DOH USING (OU_KEY)
          INNER JOIN `cl-data-procesada.AMP.DIM_DATE` DD USING (DATE_KEY)
          INNER JOIN  `cl-data-procesada.AMP.DIM_STORE` DS USING (OU_KEY)
          INNER JOIN  `cl-data-procesada.AMP.DIM_SKU_HIERARCHY` DSKU USING (SKU_KEY)
        WHERE LAOLS.DATE_VALUE >= '{current_date}' AND LAOLS.DATE_VALUE <= '{current_date}'
          AND DS.STORE_ID IN ({str_store_id})
          AND DSKU.SKU_PRODUCT IN ({products_unique_str})
        GROUP BY
          sku,
          fecha_medicion_inventario    
    """

In [170]:
from google.cloud import bigquery
from utils.environment import get_sa_json_from_consul
consul_host = 'consul.smu-labs.cl'
consul_port = 80
consul_token = 'ae3fe4c9-5f9e-6fe3-2e8b-2d012ea3168e'
consul_sa_file = 'smu-sip/smu-sip-profile-matrix/labs/service-account.json'
sa_json = get_sa_json_from_consul(consul_host, consul_port, consul_token, consul_sa_file)

In [171]:
client = bigquery.Client.from_service_account_info(sa_json)
query_job = client.query(stock_query)
stock_data = query_job.to_dataframe()
stock_data.columns = ['sku', 'predictionday', 'stock_umb']
stock_data['sku'] = stock_data['sku'].astype(int)
stock_data['stock_umb'] = stock_data['stock_umb'].astype(int)
stock_data

,sku,predictionday,stock_umb
0,966679,2023-12-01,22
1,655129,2023-12-01,0
2,571581,2023-12-01,0
3,655130,2023-12-01,0
4,636093,2023-12-01,42
5,631234,2023-12-01,0
6,966668,2023-12-01,303
7,858091,2023-12-01,0
8,638484,2023-12-01,0
9,626218,2023-12-01,32


In [172]:
stock_data['predictionday'] = pd.to_datetime(stock_data['predictionday'], format='%Y-%m-%d')

bakery['sku'] = bakery['sku'].astype(int)
bakery['predictionday'] = pd.to_datetime(bakery['predictionday'], format='%Y-%m-%d')

roastery['sku'] = roastery['sku'].astype(int)
roastery['predictionday'] = pd.to_datetime(roastery['predictionday'], format='%Y-%m-%d')

C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_17168\1344997064.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bakery['sku'] = bakery['sku'].astype(int)
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_17168\1344997064.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bakery['predictionday'] = pd.to_datetime(bakery['predictionday'], format='%Y-%m-%d')
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_17168\1344997064.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [173]:
bakery_merge = bakery.merge(stock_data, on=['sku', 'predictionday'], how='left')
bakery_merge

,sku_name,sku,group_id,type,store_id,predictionday,referenceday,finalforecast,stock_umb
0,PAN COLIZA CORRIENTE AMADA MASA KG,626218,170020302,1,67,2023-12-01,2023-12-01,3.0,32
1,PAN HALLULLA PLANA LAF KG,632861,170020104,1,67,2023-12-01,2023-12-01,1.0,35
2,PAN CIABATTA AMADA MASA KG,638771,170020302,1,67,2023-12-01,2023-12-01,22.0,110
3,PAN HALLULLA AMADA MASA KG,966668,170020104,1,67,2023-12-01,2023-12-01,63.0,303
4,PAN MARRAQUETA AMADA MASA KG,966669,170020102,1,67,2023-12-01,2023-12-01,71.0,389
5,PAN DOBLADA AMADA MASA KG,966670,170020302,1,67,2023-12-01,2023-12-01,9.0,65
6,PAN HALLULLA INTEGRAL AMADA MASA KG,966679,170020302,1,67,2023-12-01,2023-12-01,3.0,22


In [174]:
roastery_merge = roastery.merge(stock_data, on=['sku', 'predictionday'], how='left')
roastery_merge

,sku_name,sku,group_id,type,store_id,predictionday,referenceday,finalforecast,stock_umb
0,POLLO ASADO LISTO TRADICIONAL TM UN.,655130,172011201,2,67,2023-12-01,2023-12-01,6.0,0


In [175]:
bakery_prediction = bakery_merge.copy()
roastery_prediction = roastery_merge.copy()

In [176]:
dim_products = pd.read_csv('SOURCES_sip_dim_products.csv', sep=',')
dim_products

,sku,sku_name,group_name,group_id,category_name,category_id,unit_measurement,type,sku_sales
0,858091,POLLO ASADO F CAL ELAB PROPIA UN,POLLO ASADO ENTERO,172011201,POLLOS ASADOS,1720112,ST,2,612745
1,638484,POLLO ASADO C ELAB PROPIA UN,POLLO ASADO ENTERO,172011201,POLLOS ASADOS,1720112,ST,2,612746
2,626218,PAN COLIZA CORRIENTE AMADA MASA KG,PAN ESPECIAL TERCERO,170020302,PAN ESPECIAL,1700203,KG,1,626218
3,632861,PAN HALLULLA PLANA LAF KG,HALLULLA TERC,170020104,PAN CORRIENTE,1700201,KG,1,632861
4,638771,PAN CIABATTA AMADA MASA KG,PAN ESPECIAL TERCERO,170020302,PAN ESPECIAL,1700203,KG,1,638771
5,571581,POLLO ASADO CONG ELAB PROPIA UN,POLLO ASADO ENTERO,172011201,POLLOS ASADOS,1720112,ST,2,642541
6,655130,POLLO ASADO LISTO TRADICIONAL TM UN.,POLLO ASADO ENTERO,172011201,POLLOS ASADOS,1720112,ST,2,665561
7,966668,PAN HALLULLA AMADA MASA KG,HALLULLA TERC,170020104,PAN CORRIENTE,1700201,KG,1,966668
8,966669,PAN MARRAQUETA AMADA MASA KG,MARRAQUETA TERC,170020102,PAN CORRIENTE,1700201,KG,1,966669
9,966670,PAN DOBLADA AMADA MASA KG,PAN ESPECIAL TERCERO,170020302,PAN ESPECIAL,1700203,KG,1,966670


In [177]:
bakery_detailed_data = pd.read_csv('SOURCES_sip_dim_bakery_production.csv', sep=',')
bakery_detailed_data

,sku_product,product_nm,unit_weight,units_per_tray,baking_minutes
0,626218,PAN COLIZA CORRIENTE AMADA MASA KG,110,15,10
1,966676,PAN HALLULLA ESPECIAL TRC KG,60,24,10
2,966679,PAN HALLULLA INTEGRAL AMADA MASA KG,100,15,10
3,966668,PAN HALLULLA AMADA MASA KG,100,15,10
4,635098,PAN HALLULLA INTEGRAL LINAZA LAF KG,60,15,10
5,966675,PAN CHOCOSO TRC KG,85,15,10
6,638771,PAN CIABATTA AMADA MASA KG,105,12,10
7,605439,PAN MARRAQUETA INTEGRAL AMADA MASA KG,200,8,10
8,966669,PAN MARRAQUETA AMADA MASA KG,200,8,10
9,638770,PAN TOSCANO KG,70,20,10


In [178]:
roastery_detailed_data = pd.read_csv('SOURCES_sip_dim_roastery_production.csv', sep=',')
roastery_detailed_data

,store_id,oven_model,oven_brand,operative_ovens,energy_source,oven_type,accessories_type,qty_accessories,qty_inputs_per_accessory,maintainers,qty_maintainers
0,67,SR7-PTF,Macadams,1,Gas,Rosticero,Pincho,3,7,1,20
1,328,HR7,Hobart,1,Electrico,Rosticero,Pincho,3,7,1,24
2,332,STG7,Fri-jado,1,Electrico,Rosticero,Pincho,3,7,1,24
3,486,STG7,Fri-jado,1,Electrico,Rosticero,Pincho,3,7,1,20
4,978,SelfCookingCenter 10x1/1 GN,Rational,1,Electrico,Cominado,Superspike,3,6,1,20


In [195]:
profile_df = pd.read_csv('TRANSFORMED_67_profile_matrix.csv', sep=',')
profile_df

,store_id,sku_product,week,weekday,interval,records_sum,number,day_total,weight
0,67,612746,49,5,13:00:00 - 13:09:59,0.250000,37,0.250000,1.000000
1,67,626218,48,0,10:00:00 - 10:09:59,0.236364,19,27.909091,0.008469
2,67,626218,48,0,10:50:00 - 10:59:59,0.236364,24,27.909091,0.008469
3,67,626218,48,0,12:40:00 - 12:49:59,1.386364,35,27.909091,0.049674
4,67,626218,48,0,13:10:00 - 13:19:59,0.672727,38,27.909091,0.024104
...,...,...,...,...,...,...,...,...,...
4531,67,966679,51,6,14:00:00 - 14:09:59,1.010000,43,51.745000,0.019519
4532,67,966679,51,6,15:00:00 - 15:09:59,2.530000,49,51.745000,0.048894
4533,67,966679,51,6,17:20:00 - 17:29:59,1.935000,63,51.745000,0.037395
4534,67,966679,51,6,17:30:00 - 17:39:59,1.000000,64,51.745000,0.019326


In [212]:
bakery['predictionday'].dt.weekday

11     4
43     4
57     4
119    4
142    4
165    4
198    4
Name: predictionday, dtype: int32

In [200]:
bakery_unique_list = bakery['sku'].unique().tolist()
bakery_unique_list

[626218, 632861, 638771, 966668, 966669, 966670, 966679]

In [238]:
weekday_series = pd.to_datetime(bakery['predictionday']).dt.weekday
weekday_series[0]

KeyError: 0

In [239]:
bakery_prediction = profile_df.loc[(profile_df['sku_product'].isin(bakery_unique_list)) & (profile_df['weekday']==pd.to_datetime(bakery['predictionday']).dt.weekday.iloc[0])]
#bakery_prediction1 = profile_df.loc[(profile_df['sku_product'].isin(bakery_unique_list))]
bakery_prediction

,store_id,sku_product,week,weekday,interval,records_sum,number,day_total,weight
23,67,626218,48,4,16:30:00 - 16:39:59,2.436364,58,36.509091,0.066733
24,67,626218,48,4,17:30:00 - 17:39:59,0.795455,64,36.509091,0.021788
25,67,626218,48,4,21:00:00 - 21:09:59,1.072727,85,36.509091,0.029382
26,67,626218,48,4,21:10:00 - 21:19:59,0.704545,86,36.509091,0.019298
63,67,626218,49,4,11:50:00 - 11:59:59,1.450000,30,36.509091,0.039716
...,...,...,...,...,...,...,...,...,...
4510,67,966679,51,4,14:20:00 - 14:29:59,0.510000,45,37.420000,0.013629
4511,67,966679,51,4,16:50:00 - 16:59:59,0.760000,60,37.420000,0.020310
4512,67,966679,51,4,17:30:00 - 17:39:59,0.725000,64,37.420000,0.019375
4513,67,966679,51,4,17:40:00 - 17:49:59,1.000000,65,37.420000,0.026724


In [240]:
def production_batches(df :pd.DataFrame, products :pd.DataFrame, predictions :pd.DataFrame, detail :pd.DataFrame) -> pd.DataFrame:
    # Variable to know which process will be evaluated
    # renombrar columna 'sku_product' por 'sku'
    df.rename(columns={'sku_product':'sku'}, inplace=True)
    detail.rename(columns={'sku_product':'sku'}, inplace=True)
    classification_state = 0

    # Identify process, bakery type 1 and roast chiken type 2
    # global classification_state
    if not (df.loc[df['sku'].isin(products.loc[products['type']==1]['sku_sales'].unique().tolist())].empty):
        classification_state = 1
        df = df.loc[df['sku'].isin(products.loc[products['type']==1]['sku_sales'].unique().tolist())]
    elif not (df.loc[df['sku'].isin(products.loc[products['type']==2]['sku_sales'].unique().tolist())].empty):
        classification_state = 2
        df = df.loc[df['sku'].isin(products.loc[products['type']==2]['sku_sales'].unique().tolist())]
    
    # Create bins
    bin_edges = [i for i in range(0, 93, 3)]
    
    # Calculates the sum of peso for each number range in relation to the products column.
    df['intervalo_'] = pd.cut(df['number'], bins=bin_edges, right=True)
    df = df.groupby(['sku', 'intervalo_'])['weight'].sum().reset_index()
    df['weight'].fillna(0, inplace=True)

    # Pivot the table to get the calculations as columns
    df = df.pivot_table(index='sku', columns='intervalo_', values='weight', fill_value=0)

    # Renames columns according to interval schedule 
    df.columns = [7 + 0.5 * i for i in range(len(df.columns))]
    df.reset_index(inplace=True) 
    
    # Identify the process and perform a merge with the predictions
    if classification_state == 1:
        df = df.loc[df['sku'].isin(products.loc[products['type']==1]['sku_sales'].unique().tolist())]
        df = df.merge(predictions, on=['sku'], how='inner')
    elif classification_state == 2:
        df = df.loc[df['sku'].isin(products.loc[products['type']==2]['sku_sales'].unique().tolist())]
        df = df.merge(predictions, on=['sku'], how='inner')

    # Convert the values from percentages to target value per interval
    hourly_columns = [i/2 + 7 for i in range(30)]
    df[hourly_columns] = df[hourly_columns].multiply(df['finalforecast'], axis=0)
    
    # Evaluate process for establishing the columns to be used
    if classification_state == 1:
        keep_columns = ['sku', 'unit_weight', 'units_per_tray']
        bakery_columns = keep_columns + hourly_columns
        df = df.merge(detail, on = 'sku', how='left')[bakery_columns]
    elif classification_state == 2:
        keep_columns = ['sku', 'qty_accessories', 'qty_inputs_per_accessory']
        roast_chiken_columns = keep_columns + hourly_columns
        df = df.merge(detail, on = 'store_id', how='left')[roast_chiken_columns]
         
                
    return df


In [219]:
bakery_batches = {0:[8, 11],
                1:[11, 14],
                2:[14, 17],
                3:[17, 21]}

roast_chiken_batches = {0:[8, 12],
                        1:[12, 21]} 

bakery_families = {1 : [626218, 966679, 966668],
                   2 : [638771, 966669],
                   3 : [966670, 632861]}

In [232]:

df_bakery = production_batches(bakery_prediction, dim_products, bakery, bakery_detailed_data)
df_bakery

C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_17168\3215818411.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'sku_product':'sku'}, inplace=True)
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_17168\3215818411.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = df.groupby(['sku', 'intervalo_'])['weight'].sum().reset_index()


,sku,unit_weight,units_per_tray,7.0,7.5,8.0,8.5,9.0,9.5,10.0,10.5,11.0,11.5,12.0,12.5,13.0,13.5,14.0,14.5,15.0,15.5,16.0,16.5,17.0,17.5,18.0,18.5,19.0,19.5,20.0,20.5,21.0,21.5
0,626218,110,15,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.119148,0.130727,0.000000,0.128860,0.000000,0.098232,0.000000,0.000000,0.000000,0.000000,0.471365,0.000000,0.185632,0.189741,0.000000,0.546066,0.348481,0.251370,0.384338,0.146041,0.0
1,632861,100,15,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.213805,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.213357,0.000000,0.000000,0.264904,0.000000,0.000000,0.176602,0.000000,0.131331,0.000000,0.0
2,638771,105,12,0.0,0.0,0.000000,0.596910,0.475386,0.845310,0.689524,0.990388,1.212556,0.494659,0.729140,0.981287,1.149386,1.157951,0.688989,0.301935,0.312106,0.593698,0.330308,1.183648,1.134931,1.318555,1.550359,2.937973,1.937949,0.277844,0.000000,0.000000,0.109210,0.0
3,966668,100,15,0.0,0.0,0.219303,0.336116,1.519458,1.073691,1.275092,1.191846,1.102334,1.792916,1.459038,0.883926,1.444716,1.548549,2.480812,1.659991,1.962540,0.977018,1.143957,3.723232,3.827065,2.841991,3.790813,6.720973,7.646522,2.462462,4.975946,3.906730,1.032963,0.0
4,966669,200,8,0.0,0.0,0.000000,0.473218,2.430337,1.543836,3.061425,1.441593,1.005981,3.260428,2.516911,3.864487,2.474212,1.725210,2.600743,2.176491,4.384322,2.029198,1.326814,3.623568,2.566662,3.204801,4.205690,5.173280,4.600952,5.597923,2.763314,2.892979,0.055627,0.0
5,966670,80,20,0.0,0.0,0.000000,0.227290,0.852632,0.527096,0.198441,0.128655,0.211696,0.417154,0.364522,0.439766,0.147758,0.224951,0.349318,0.161014,0.521637,0.109552,0.250292,0.541520,0.415205,0.492398,0.938791,0.173489,0.311111,0.039766,0.516569,0.075634,0.363743,0.0
6,966679,100,15,0.0,0.0,0.000000,0.060529,0.060128,0.138295,0.000000,0.082977,0.194415,0.000000,0.141502,0.161545,0.143105,0.121058,0.142304,0.101416,0.000000,0.115446,0.000000,0.118653,0.193212,0.376403,0.381213,0.000000,0.151924,0.236505,0.000000,0.000000,0.079369,0.0
